In [32]:
import json
import re
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB

In [20]:
nl_f = open('../data/nl.json')
en_f = open('../data/en.json')
nl_data = json.load(nl_f)
en_data = json.load(en_f)

In [21]:
urls_tokenized = []
excluded_keywords = ['www', 'index', 'html', 'htm', 'html', 'http', 'https']
nl_tlds = ['.nl/', '.be/', '.su/', '.aw/', '.sx/', '.cw/']
other_tlds = ['.com/', '.net/', '.org/']

for element in nl_data:
    url_element = element['siteUrl'] + ('' if element['siteUrl'].endswith('/') else '/')
    split = [word for word in re.split('[^a-zA-Z]', url_element) 
        if len(word) >= 2 and word not in excluded_keywords]
    custom_feat =[
        1 if any(tld in url_element for tld in nl_tlds) else 0,
        1 if any(tld in url_element for tld in other_tlds) else 0
    ]
    urls_tokenized.append([split, custom_feat, url_element, 1])

for element in en_data:
    url_element = element['siteUrl'] + ('' if element['siteUrl'].endswith('/') else '/')
    split = [word for word in re.split('[^a-zA-Z]', url_element) 
        if len(word) >= 2 and word not in excluded_keywords]
    custom_feat =[
        1 if any(tld in url_element for tld in nl_tlds) else 0,
        1 if any(tld in url_element for tld in other_tlds) else 0
    ]
    urls_tokenized.append([split, custom_feat, url_element, 0])

In [22]:
print(urls_tokenized[0])

[['tevansleen', 'nl'], [1, 0], 'https://www.tevansleen.nl/', 1]


In [5]:
# NOTE: NOT NECESSARY SINCE TRAIN_TEST_SPLIT ALREADY SHUFFLES VALUES

#Train Test split - ONLY RUN ONCE AFTER INITIALIZATION
# random.shuffle(urls_tokenized)

In [35]:
X_tokens, X_url, y = [], [], []
for url in urls_tokenized:
    X_tokens.append(url[0])
    X_url.append(url[2])
    y.append(url[3])

print(X_tokens[0], y[0])

vectorizer = CountVectorizer()
X_tokens = vectorizer.fit_transform(X_url)

print(X_tokens)

['tevansleen', 'nl'] 1
  (0, 48496)	1
  (0, 113266)	1
  (0, 97817)	1
  (0, 70871)	1


In [41]:
print(vectorizer.get_feature_names_out()[80000])

primhome


In [24]:
X_token_train, X_token_test, y_token_train, y_token_test = train_test_split(X_tokens, y, test_size=0.2)

In [25]:
# ccTLD - no training necessary because it just looks at TLD:
ccTLD_y = []
for url in X_url:
    ccTLD_y.append(1 if any(tld in url for tld in nl_tlds) else 0)

print(len(ccTLD_y))

# ccTLD+ Would give the same outcome as TLD in our case because it counts generic TLD's (.com/.net/.org) as non-dutch anyway.

108462


In [29]:
print(X_token_train[0])
print(y_token_train[0])

['torstenkeller', 'com']
0


In [28]:
# Token features



gnb = GaussianNB()
y_pred = gnb.fit(X_token_train, y_token_train).predict(X_token_test)

/Users/bonoijpelaar/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:738: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array = np.asarray(array, order=order, dtype=dtype)


ValueError: Expected 2D array, got 1D array instead:
array=[list(['torstenkeller', 'com']) list(['barchem', 'co', 'uk'])
 list(['haro', 'online', 'com', 'movies', 'agent', 'cody', 'banks']) ...
 list(['peakuplink', 'com']) list(['robbiehaf', 'com', 'Recipes'])
 list(['brikat', 'be', 'nieuw'])].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
# Custom features

In [27]:
confusion_matrix(y, ccTLD_y)

array([[58975,   380],
       [ 8386, 40721]])